In [118]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,r2_score

In [2]:
from google.colab import files
u = files.upload()

Saving CarPricesPrediction.csv to CarPricesPrediction.csv


In [155]:
data = pd.read_csv("CarPricesPrediction.csv")
data.drop("Unnamed: 0",axis=1,inplace=True)
data.head(-1)

,Make,Model,Year,Mileage,Condition,Price
0,Ford,Silverado,2022,18107,Excellent,19094.75
1,Toyota,Silverado,2014,13578,Excellent,27321.10
2,Chevrolet,Civic,2016,46054,Good,23697.30
3,Ford,Civic,2022,34981,Excellent,18251.05
4,Chevrolet,Civic,2019,63565,Excellent,19821.85
...,...,...,...,...,...,...
994,Honda,Silverado,2015,50874,Excellent,24456.40
995,Nissan,Camry,2010,149032,Excellent,24548.50
996,Chevrolet,F-150,2014,20608,Excellent,26969.70
997,Ford,Altima,2016,109851,Good,20507.55


In [35]:
data.isna().sum()

Make         0
Model        0
Year         0
Mileage      0
Condition    0
Price        0
dtype: int64

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Make       1000 non-null   object 
 1   Model      1000 non-null   object 
 2   Year       1000 non-null   int64  
 3   Mileage    1000 non-null   int64  
 4   Condition  1000 non-null   object 
 5   Price      1000 non-null   float64
dtypes: float64(1), int64(2), object(3)
memory usage: 47.0+ KB


In [156]:
data["Year"] = data["Year"].astype(str)

In [157]:
for i in data.columns:
  print(f"Column {i}: {data[i].unique()}")

Column Make: ['Ford' 'Toyota' 'Chevrolet' 'Nissan' 'Honda']
Column Model: ['Silverado' 'Civic' 'Altima' 'Camry' 'F-150']
Column Year: ['2022' '2014' '2016' '2019' '2013' '2015' '2012' '2021' '2017' '2020'
 '2010' '2018' '2011']
Column Mileage: [ 18107  13578  46054  34981  63565  23600  11470  89471 121298 110691
 112693 101914  79902  97489  72976  40633  48281  33054 121446  61672
 126395 119784 148697 100665  66951  65559  69109  75631  34080 134672
  54295  20693  63740 117503  81466 106330  43785 102118  96202 133627
 131994  38705 139945  69108  17149  53165 105649  72408  88835  17332
 123884  40238  36817  44880  77083  33565  36404  81972  90846  11809
  19317  54542  16493  99506  38647 103960 132338  68639  56311 102278
  35930  96869  87051  46163 119991 135196  23116  66236 144751 101000
  27511  14488 105493  58378 119427 119802 141113 146622  79364 141793
  14744  70347  68245  83613 133106 119617  57207  52477  37978  53814
  17346  44885  50778  70039  89140  43774  22

In [158]:
ohe = OneHotEncoder(sparse_output=False).set_output(transform="pandas")
encoded_data = ohe.fit_transform(data[["Make","Model"]])
data = pd.concat([data,encoded_data],axis=1).drop(["Make","Model"],axis=1)

In [159]:
a = sorted(['2022','2014', '2016', '2019', '2013', '2015', '2012', '2021', '2017', '2020',
 '2010', '2018', '2011'])
a

['2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022']

In [193]:
x = data.drop("Price",axis=1)
y = data["Price"]

In [194]:
encoder = OrdinalEncoder(categories=[
    ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022'],
    ['Fair','Good', 'Excellent']])

In [195]:
x1 = x[["Year","Condition"]]
x1

,Year,Condition
0,2022,Excellent
1,2014,Excellent
2,2016,Good
3,2022,Excellent
4,2019,Excellent
...,...,...
995,2010,Excellent
996,2014,Excellent
997,2016,Good
998,2010,Good


In [196]:
x.head()

,Year,Mileage,Condition,Make_Chevrolet,Make_Ford,Make_Honda,Make_Nissan,Make_Toyota,Model_Altima,Model_Camry,Model_Civic,Model_F-150,Model_Silverado
0,2022,18107,Excellent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2014,13578,Excellent,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,2016,46054,Good,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,2022,34981,Excellent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,2019,63565,Excellent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [197]:
x1 = encoder.fit_transform(x1)
x1 = pd.DataFrame(x1)

In [198]:
x = pd.concat([x,x1],axis=1).drop(["Year","Condition"],axis=1)

In [332]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)

In [333]:
x_train.columns = x_train.columns.astype(str)
x_test.columns = x_test.columns.astype(str)

In [334]:
scl = MinMaxScaler()
x_train = scl.fit_transform(x_train)
x_test = scl.fit_transform(x_test)
x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)

In [335]:
x_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.891734,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.083333,0.0
1,0.116686,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.416667,0.0
2,0.662271,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.250000,1.0
3,0.650357,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,1.0
4,0.415532,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,1.0


In [342]:
GD = SGDRegressor(loss='squared_epsilon_insensitive',penalty="l1",alpha=0.001,learning_rate='optimal',eta0=0.01,l1_ratio=0.15,fit_intercept=True,max_iter=1000)
GD.fit(x_train,y_train)
GD_pred = (GD.predict(x_test))

In [343]:
print(f"Linear model accuracy: {r2_score(y_test,GD_pred)}")

Linear model accuracy: 0.999994004958097


In [352]:
GDFrame = pd.DataFrame(GD_pred)
predicted_labels = pd.concat([x_test,GDFrame],axis=1)
predicted_labels.columns = data.columns
predicted_labels

,Year,Mileage,Condition,Price,Make_Chevrolet,Make_Ford,Make_Honda,Make_Nissan,Make_Toyota,Model_Altima,Model_Camry,Model_Civic,Model_F-150,Model_Silverado
0,0.141447,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,1.0,30506.162059
1,0.288088,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.416667,0.0,24482.087846
2,0.005841,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.083333,1.0,30453.186637
3,0.817832,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.083333,1.0,24782.514725
4,0.800158,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.333333,1.0,21905.957274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.057585,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.916667,1.0,20091.867628
196,0.628136,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.750000,0.0,18107.318049
197,0.062966,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.333333,0.5,27054.240763
198,0.399247,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.666667,1.0,20705.793663


In [290]:
param_grid = {
    'loss': ['squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
    'penalty': ['none', 'l2', 'l1', 'elasticnet'],
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
    'eta0': [0.01, 0.1, 0.5],
    'max_iter': [1000, 2000, 3000],
}

In [263]:
grid = GridSearchCV(estimator=SGDRegressor(), param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid.fit(x_train,y_train)
grid.best_params_

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/py

{'alpha': 0.001,
 'eta0': 0.01,
 'learning_rate': 'optimal',
 'loss': 'squared_epsilon_insensitive',
 'max_iter': 1000,
 'penalty': 'l1'}